# Analysis Code for 2-Point Correlator

## Preparation

In [1]:
%pip install numpy scipy iminuit matplotlib scienceplots

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
from iminuit import Minuit
from iminuit.cost import LeastSquares
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('retina')
from matplotlib import ticker
import scienceplots

## Potential

## Effective mass

### Figure

In [14]:
plt.style.use(["science", "nature"])

tsize = 64
cutoff = 2.1753
homePath = os.getenv("HOME")
codeRoot = "{}/ccbar".format(homePath)


def gauge_plot(coulomb, landau, fit, filename, cutoff=1.0, xrange=None, yrange=None, loc=8):
    fig, ax = plt.subplots()

    errbar_plot_style1 = {
        "fmt": "D",
        "color": "xkcd:primary blue",
        "markersize": 4,
        "markeredgewidth": 0.5,
        "linewidth": 0.5,
        "fillstyle": "none",
    }

    errbar_plot_style2 = {
        "fmt": "s",
        "color": "xkcd:bright red",
        "markersize": 4,
        "markeredgewidth": 0.5,
        "linewidth": 0.5,
        "fillstyle": "none",
    }

    legend_style = {
        "loc": loc,
        "handletextpad": 0.5,
        "labelspacing": 0.3,
        "fontsize": 11
    }

    index = np.arange(0, tsize, 1)
    ax.errorbar(
        index + 0.5,
        coulomb[:, 1] * cutoff,
        coulomb[:, 2] * cutoff,
        label="Coulomb",
        **errbar_plot_style1,
    )
    ax.errorbar(
        index + 0.5,
        landau[:, 1] * cutoff,
        landau[:, 2] * cutoff,
        label="Landau",
        **errbar_plot_style2,
    )
    x_fit = np.arange(0, 32, 0.01)
    y_fit = np.full(x_fit.shape, fit)
    ax.plot(
            x_fit,
            y_fit,
            linewidth=1,
            color="black",
            # label="fit",
        )


    # Set grid (reserved)
    ax.grid(which="major", color="#DDDDDD", linewidth=0.5)
    ax.grid(which="minor", color="#EEEEEE", linestyle=":", linewidth=0.5)

    ax.legend(**legend_style)

    ax.set_xlabel(r"$t/a$")
    ax.xaxis.set_major_locator(ticker.MultipleLocator(5))
    if xrange is not None:
        ax.set_xlim(xrange[0], xrange[1])

    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
    ax.set_ylabel(r"$m_{\rm eff}\ [{\rm GeV}]$")
    if yrange is not None:
        ax.set_ylim(yrange[0], yrange[1])

    fig.savefig("{}.png".format(filename), dpi=600)
    plt.close()


# Channel
channel = ["ps", "v", "s", "av", "t"]

emass_c, emass_l = [[] for _ in range(2)]  # Read data files
for i in range(5):
    emass_c.append(
        np.loadtxt("{}/result/c2pt/effmass/exp.{}.txt".format(codeRoot, channel[i]))
    )
    emass_l.append(
        np.loadtxt("{}/result/l2pt/effmass/exp.{}.txt".format(codeRoot, channel[i]))
    )

xrange_all = [[4, 29], [4, 29]]
yrange_all = [[2.7, 3.1], [2.7, 3.1]]
loc = [8, 8]
Mfit = [2.9725, 3.0777]

# Destination
path = f"{homePath}/lqcd/ccbar:analysis/fig/effmass"
if not os.path.exists(path):
    os.makedirs(path)

# PLOT
for i in range(2):
    gauge_plot(
        emass_c[i],
        emass_l[i],
        Mfit[i],
        "{}/gauge_{}_wfit".format(path, channel[i]),
        cutoff=cutoff,
        xrange=xrange_all[i],
        yrange=yrange_all[i],
        loc=loc[i],
    )
